# My Update
In this notebook I will expand the functionalities of the current RAG system implemented together with my group.

In [1]:
import json

from rag_service.rag_chat import rag_interaction

## Knowledge Base
Instead of getting the Knowledge Base data through the API, as we did for the project, I downloaded the data so I could have it locally and not have to log in every time to get it. The data in the KB consists of two JSON files that contain the description of the available KPIs and the list of machines with the corresponding KPIs.

In [2]:
file_path = "data/kpi_data.json"

with open(file_path, "r") as file:
    kpi_data = json.load(file)

In [3]:
file_path = "data/machine_data.json"

with open(file_path, "r") as file:
    machine_data = json.load(file)

In [4]:
data = [kpi_data, machine_data]

## Queries

In [5]:
query = "How many machines are there?"
response = rag_interaction(data, query)

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
print(response)

There are various types of machines represented in the knowledge base, including those related to energy consumption and production. However, without further clarification on the specific type of machine or category, it's challenging to provide an exact number. If you could provide additional context or details about the type of machine being queried, I would be happy to try and assist further.


---

In [7]:
query = "Can you speak italian?"
response = rag_interaction(data, query)

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
print(response)

Sì, posso fornire informazioni sulla lingua italiana.


---

In [9]:
query = "Show me the KPIs for the Large Capacity Cutting Machine 1"
response = rag_interaction(data, query)

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/7 [00:00<?, ?it/s]

In [10]:
print(response)

Unfortunately, I don't have information on the specific KPIs for the Large Capacity Cutting Machine 1. It's possible that this data is not available in my knowledge base or is not relevant to my current domain expertise. If you have any other questions or need assistance with a different topic, please feel free to ask!


---

In [11]:
query = "Generate a new KPI"
response = rag_interaction(data, query)

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/7 [00:00<?, ?it/s]

In [12]:
print(response)

{
  "KPIs": [
    {
      "name": "Cycle Efficiency",
      "type": "Ratio",
      "description": "Calculates the efficiency of the machine in completing cycles within a specified time frame.",
      "unit_of_measure": "%",
      "formula": "(Mean Cycle Time / Target Cycle Time) * 100"
    },
    {
      "name": "Production Rate",
      "type": "Quantity",
      "description": "Calculates the rate at which the machine produces units within a specified time frame.",
      "unit_of_measure": "units/second",
      "formula": "(Total Units Produced / Total Time Elapsed) * 60"
    }
  ]
}


## Exploit the Memory Functionality

### Dynamic Reports

### Error Correction

### Personalized Recommendations